# 1 Mechine Learning

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("data/titanic.csv", index_col="PassengerId")
df.drop(columns=["Name","Ticket","Age","Cabin"],inplace=True)

In [26]:
# X = input dan y = Target

X = df.drop(columns=["Survived"])
y = df["Survived"]

#split data                  
#stratify berguna untuk mengklasifikasikan berdasarkan label survive dan not survive
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 6), (179, 6), (712,), (179,))

## Prepocessor Menggunakan Pipe line Agar data tidak bocor / data leakage

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

In [5]:
numerical_pipeline = Pipeline([
    #nama steps 
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", MinMaxScaler())
])

categorical_pipeline = Pipeline([ 
    ("imputer", SimpleImputer(strategy="most_frequent")),
    #encode
    ("onehot", OneHotEncoder())
])

In [7]:
X_train.head()

,Pclass,Sex,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,
693,3,male,0,0,56.4958,S
482,2,male,0,0,0.0000,S
528,1,male,0,0,221.7792,S
856,3,female,0,1,9.3500,S
802,2,female,1,1,26.2500,S


In [15]:
#berguna untuk memfilter kolom mana yang masuk pipa numerik dan pipa kategori
from sklearn.compose import ColumnTransformer
preprocesor = ColumnTransformer([
    ("numeric", numerical_pipeline, ["SibSp","Parch","Fare"]),
    ("categoric",categorical_pipeline,["Pclass","Sex","Embarked"])
])

### Pipeline

In [16]:
from sklearn.neighbors import KNeighborsClassifier

pipeline = Pipeline([
    ("prep", preprocesor),
    ("algo",KNeighborsClassifier())
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['SibSp', 'Parch', 'Fare']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Pclass', 'Sex',
                           

In [17]:
pipeline.score(X_test, y_test)

0.776536312849162

# 2 Grid Search CV(Cross Validation) ---Final---

In [18]:
from sklearn.model_selection import GridSearchCV

#paremeter tuning
parameter = {
    #kenapa ada algo karna untuk memberi tahu n_neigborh punya nya pipeline KNeighborsClassifier()
    "algo__n_neighbors":range(1,51,2),
    "algo__weights":["uniform","distance"],
    "algo__p":[1,2]
}

#biasany crosvalidation 3-5
#n_jobs mw berapa banyak core cpu di pakai
model = GridSearchCV(pipeline, parameter,cv=3,n_jobs=-1,verbose =1)
model.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['SibSp',
                                                                          'Parch',
                                                                          'Fare']),
                                                                        ('categoric',
                                                                         Pipeline(steps=[('impu

In [19]:
pd.DataFrame(model.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algo__n_neighbors,param_algo__p,param_algo__weights,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
36,0.028880,0.003545,0.045320,0.015789,19,1,uniform,"{'algo__n_neighbors': 19, 'algo__p': 1, 'algo_...",0.815126,0.818565,0.805907,0.813200,0.005344,1
28,0.030579,0.007441,0.045260,0.008817,15,1,uniform,"{'algo__n_neighbors': 15, 'algo__p': 1, 'algo_...",0.815126,0.810127,0.810127,0.811793,0.002357,2
40,0.020781,0.001580,0.031566,0.000417,21,1,uniform,"{'algo__n_neighbors': 21, 'algo__p': 1, 'algo_...",0.815126,0.818565,0.801688,0.811793,0.007282,2
54,0.035464,0.010580,0.047075,0.006995,27,2,uniform,"{'algo__n_neighbors': 27, 'algo__p': 2, 'algo_...",0.815126,0.827004,0.793249,0.811793,0.013981,2
38,0.028823,0.008753,0.040950,0.007347,19,2,uniform,"{'algo__n_neighbors': 19, 'algo__p': 2, 'algo_...",0.815126,0.814346,0.805907,0.811793,0.004174,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,0.027898,0.003927,0.016988,0.001234,3,1,distance,"{'algo__n_neighbors': 3, 'algo__p': 1, 'algo__...",0.760504,0.734177,0.759494,0.751392,0.012179,95
3,0.021765,0.000979,0.013981,0.000431,1,2,distance,"{'algo__n_neighbors': 1, 'algo__p': 2, 'algo__...",0.768908,0.670886,0.734177,0.724657,0.040579,97
2,0.020966,0.000241,0.030714,0.001670,1,2,uniform,"{'algo__n_neighbors': 1, 'algo__p': 2, 'algo__...",0.768908,0.670886,0.734177,0.724657,0.040579,97
1,0.031265,0.005199,0.020930,0.004780,1,1,distance,"{'algo__n_neighbors': 1, 'algo__p': 1, 'algo__...",0.764706,0.670886,0.725738,0.720443,0.038484,99


In [20]:
model.best_params_

{'algo__n_neighbors': 19, 'algo__p': 1, 'algo__weights': 'uniform'}

In [27]:
#hasil akhir yang di peroleh
#tidak ada data lekage semua aman dengan menggunakan GridSearchCV
model.score(X_train, y_train),  model.score(X_test, y_test), model.best_score_

(0.8188202247191011, 0.7877094972067039, 0.8131995414199436)

# 2 Cek prediksi melalui model yang telah kita buat

In [49]:
X.iloc[:1]

,Pclass,Sex,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,
1,3,male,1,0,7.25,S


In [114]:
data = [
    [1,"female",1,1,80,"S"],
    [3, "male",0,0,5,"S"],
    [1, "male",0,0,31,"S"]
]

X_predic = pd.DataFrame(data, index=["Rose", "Jack","Wak Yogi"], columns = X.columns)
X_predic

,Pclass,Sex,SibSp,Parch,Fare,Embarked
Rose,1,female,1,1,80,S
Jack,3,male,0,0,5,S
Wak Yogi,1,male,0,0,31,S


In [113]:
model.predict(X_predic)

array([1, 0, 0])

In [107]:
X_predic["Survived"] = model.predict(X_predic)
X_predic

,Pclass,Sex,SibSp,Parch,Fare,Embarked,Survived
Rose,1,female,1,1,80,S,1
Jack,3,male,0,0,5,S,0
Wak Yogi,1,male,0,0,31,S,1
